# Case Infomaz
Nome: Samuel Sales Nogueira Viana
2025

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações iniciais
sns.set_theme(style="whitegrid")
plt.style.use('ggplot')

# Função para salvar resultados em CSV
def salvar_csv(nome_arquivo, df):
    df.to_csv(f"{nome_arquivo}.csv", index=False)

# Função para plotar gráfico individual
def plot_grafico(df, titulo, x_label, y_label, nome_arquivo, orientacao='v'):
    plt.figure(figsize=(10, 6))
    if orientacao == 'h':
        sns.barplot(x=df[x_label], y=df[y_label])
    else:
        sns.barplot(y=df[y_label], x=df[x_label])
    plt.title(titulo)
    plt.xlabel(x_label.replace('_', ' ').title())
    plt.ylabel(y_label.replace('_', ' ').title())
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{nome_arquivo}.png", dpi=300, bbox_inches='tight')
    plt.close()

# Carregar dados do Excel
caminho_arquivo_excel = "Case_Infomaz_Base_de_Dados.xlsx"
xls = pd.ExcelFile(caminho_arquivo_excel)

# Carregar tabelas
produtos_df = pd.read_excel(xls, sheet_name="Cadastro Produtos", header=1)
clientes_df = pd.read_excel(xls, sheet_name="Cadastro Clientes", header=1)
transacoes_df = pd.read_excel(xls, sheet_name="Transações Vendas", header=1)
estoque_df = pd.read_excel(xls, sheet_name="Cadastro de Estoque", header=1)
fornecedores_df = pd.read_excel(xls, sheet_name="Cadastro Fornecedores", header=1)

# Converter datas
transacoes_df["DATA NOTA"] = pd.to_datetime(transacoes_df["DATA NOTA"])
transacoes_df["MES_ANO"] = transacoes_df["DATA NOTA"].dt.to_period('M')
estoque_df["DATA ESTOQUE"] = pd.to_datetime(estoque_df["DATA ESTOQUE"])

# Calcular VALOR UNITÁRIO apenas se não existir
if "VALOR UNITARIO" not in estoque_df.columns:
    estoque_df["VALOR UNITARIO"] = estoque_df["VALOR ESTOQUE"] / estoque_df["QTD ESTOQUE"]

# ====================
# Questão 1: Valor Total de Venda por Categoria
# ====================
df_vendas_categoria = transacoes_df.merge(
    produtos_df[["ID PRODUTO", "CATEGORIA"]], on="ID PRODUTO"
)
valor_total_venda_categoria = df_vendas_categoria.groupby("CATEGORIA")["VALOR NOTA"].sum().reset_index()
salvar_csv("questao_1_valor_total_categoria", valor_total_venda_categoria)
plot_grafico(valor_total_venda_categoria.sort_values(by="VALOR NOTA", ascending=False).head(10),
             "Questão 1 - Valor Total de Venda por Categoria",
             "VALOR NOTA", "CATEGORIA", "grafico_questao_1")

# ====================
# Questão 2: Margem dos produtos
# ====================
produtos_estoque_map = produtos_df[["ID PRODUTO", "ID ESTOQUE"]]
df_margem = transacoes_df.merge(produtos_estoque_map, on="ID PRODUTO")
df_margem = df_margem.merge(estoque_df[["ID ESTOQUE", "VALOR UNITARIO"]], on="ID ESTOQUE")
df_margem["MARGEM"] = df_margem["VALOR ITEM"] - df_margem["VALOR UNITARIO"]
margem_produtos = df_margem[["ID PRODUTO", "VALOR ITEM", "VALOR UNITARIO", "MARGEM"]].head(10)
salvar_csv("questao_2_margem_produtos", margem_produtos)

# Gráfico não recomendado — mostrar tabela

# ====================
# Questão 3: Ranking de clientes por quantidade de produtos comprados por mês
# ====================
ranking_clientes_mensal = transacoes_df.groupby(["ID CLIENTE", "DATA NOTA"])["QTD ITEM"].sum().reset_index()
ranking_clientes_mensal["MES_ANO"] = pd.to_datetime(ranking_clientes_mensal["DATA NOTA"]).dt.to_period('M')
ranking_completo = ranking_clientes_mensal.merge(clientes_df[["ID CLIENTE", "NOME CLIENTE"]], on="ID CLIENTE")
ranking_completo["POSICAO_RANKING"] = ranking_completo.groupby("MES_ANO").cumcount() + 1
top_clientes_mensal = ranking_completo.sort_values(by=["MES_ANO", "QTD ITEM"], ascending=[True, False]).head(10)
salvar_csv("questao_3_ranking_clientes", top_clientes_mensal)
plot_grafico(top_clientes_mensal,
             "Questão 3 - Top 10 Clientes por Quantidade de Produtos Comprados por Mês",
             "QTD ITEM", "NOME CLIENTE", "grafico_questao_3")

# ====================
# Questão 4: Ranking de fornecedores por quantidade de estoque por mês
# ====================
ranking_fornecedores_mensal = estoque_df.groupby(["ID FORNECEDOR", "DATA ESTOQUE"])["QTD ESTOQUE"].sum().reset_index()
ranking_fornecedores_mensal["MES_ANO"] = pd.to_datetime(ranking_fornecedores_mensal["DATA ESTOQUE"]).dt.to_period('M')
ranking_fornecedores_mensal.drop(columns=["DATA ESTOQUE"], inplace=True)
ranking_fornecedores_mensal.sort_values(by=["MES_ANO", "QTD ESTOQUE"], ascending=[True, False], inplace=True)
ranking_completo_fornecedores = ranking_fornecedores_mensal.merge(fornecedores_df[["ID FORNECEDOR", "NOME FORNECEDOR"]], on="ID FORNECEDOR")
top_fornecedores_mensal = ranking_completo_fornecedores.head(10)
salvar_csv("questao_4_ranking_fornecedores", top_fornecedores_mensal)

# Gráfico não recomendado — mostrar tabela

# ====================
# Questão 5: Ranking de produtos por quantidade de venda por mês
# ====================
ranking_produtos_mensal = transacoes_df.groupby(["ID PRODUTO", "DATA NOTA"])["QTD ITEM"].sum().reset_index()
ranking_produtos_mensal["MES_ANO"] = pd.to_datetime(ranking_produtos_mensal["DATA NOTA"]).dt.to_period('M')
ranking_completo_produtos = ranking_produtos_mensal.merge(produtos_df[["ID PRODUTO", "NOME PRODUTO", "CATEGORIA"]], on="ID PRODUTO")
ranking_completo_produtos["POSICAO_RANKING"] = ranking_completo_produtos.groupby("MES_ANO").cumcount() + 1
top_produtos_mensal = ranking_completo_produtos.sort_values(by=["MES_ANO", "QTD ITEM"], ascending=[True, False]).head(10)
salvar_csv("questao_5_ranking_produtos_quantidade", top_produtos_mensal)
plot_grafico(top_produtos_mensal,
             "Questão 5 - Top 10 Produtos por Quantidade de Venda por Mês",
             "QTD ITEM", "NOME PRODUTO", "grafico_questao_5")

# ====================
# Questão 6: Ranking de produtos por valor de venda por mês
# ====================
ranking_valor_produtos_mensal = transacoes_df.groupby(["ID PRODUTO", "DATA NOTA"])["VALOR ITEM"].sum().reset_index()
ranking_valor_produtos_mensal["MES_ANO"] = pd.to_datetime(ranking_valor_produtos_mensal["DATA NOTA"]).dt.to_period('M')
ranking_valor_produtos_mensal.rename(columns={"VALOR ITEM": "TOTAL_VENDIDO"}, inplace=True)
ranking_valor_produtos_mensal = ranking_valor_produtos_mensal.merge(produtos_df[["ID PRODUTO", "NOME PRODUTO", "CATEGORIA"]], on="ID PRODUTO")
ranking_valor_produtos_mensal["POSICAO_RANKING"] = ranking_valor_produtos_mensal.groupby("MES_ANO").cumcount() + 1
top_valor_produtos_mensal = ranking_valor_produtos_mensal.sort_values(by=["MES_ANO", "TOTAL_VENDIDO"], ascending=[True, False]).head(10)
salvar_csv("questao_6_ranking_produtos_valor", top_valor_produtos_mensal)
plot_grafico(top_valor_produtos_mensal,
             "Questão 6 - Top 10 Produtos por Valor de Venda por Mês",
             "TOTAL_VENDIDO", "NOME PRODUTO", "grafico_questao_6")

# ====================
# Questão 7: Média de valor de venda por categoria de produto por mês
# ====================
transacoes_com_categoria = transacoes_df.merge(produtos_df[["ID PRODUTO", "CATEGORIA"]], on="ID PRODUTO")
media_venda_categoria_mensal = transacoes_com_categoria.groupby(["CATEGORIA", "DATA NOTA"])["VALOR ITEM"].mean().reset_index()
media_venda_categoria_mensal["MES_ANO"] = pd.to_datetime(media_venda_categoria_mensal["DATA NOTA"]).dt.to_period('M')
media_venda_categoria_mensal.rename(columns={"VALOR ITEM": "MEDIA_VALOR_ITEM"}, inplace=True)
salvar_csv("questao_7_media_valor_categoria", media_venda_categoria_mensal)
plot_grafico(media_venda_categoria_mensal,
             "Questão 7 - Média de Valor de Venda por Categoria por Mês",
             "MEDIA_VALOR_ITEM", "CATEGORIA", "grafico_questao_7")

# ====================
# Questão 8: Ranking de margem de lucro por categoria
# ====================
df_margem_categoria = transacoes_df.merge(produtos_estoque_map, on="ID PRODUTO")
df_margem_categoria = df_margem_categoria.merge(estoque_df[["ID ESTOQUE", "VALOR UNITARIO"]], on="ID ESTOQUE")
df_margem_categoria["MARGEM"] = df_margem_categoria["VALOR ITEM"] - df_margem_categoria["VALOR UNITARIO"]
df_margem_categoria = df_margem_categoria.merge(produtos_df[["ID PRODUTO", "CATEGORIA"]], on="ID PRODUTO")
ranking_margem_categoria = df_margem_categoria.groupby(["CATEGORIA", "MES_ANO"])["MARGEM"].mean().reset_index()
ranking_margem_categoria.rename(columns={"MARGEM": "MEDIA_MARGEM"}, inplace=True)
salvar_csv("questao_8_ranking_margem_categoria", ranking_margem_categoria)
plot_grafico(ranking_margem_categoria,
             "Questão 8 - Média de Margem de Lucro por Categoria por Mês",
             "MEDIA_MARGEM", "CATEGORIA", "grafico_questao_8")

# ====================
# Questão 9: Lista de produtos comprados por clientes
# ====================
transacoes_com_produtos = transacoes_df.merge(produtos_df[["ID PRODUTO", "NOME PRODUTO", "CATEGORIA"]], on="ID PRODUTO")
transacoes_com_produtos = transacoes_com_produtos.merge(clientes_df[["ID CLIENTE", "NOME CLIENTE"]], on="ID CLIENTE")
top_produtos_cliente = transacoes_com_produtos.groupby(["NOME CLIENTE", "NOME PRODUTO"])["QTD ITEM"].sum().reset_index()
top_produtos_cliente = top_produtos_cliente.sort_values(by="QTD ITEM", ascending=False).head(20)
salvar_csv("questao_9_lista_produtos_clientes", top_produtos_cliente)

# Gráfico não recomendado — mostrar tabela

# ====================
# Questão 10: Ranking de produtos por quantidade de estoque
# ====================
estoque_com_produtos = estoque_df.merge(produtos_df[["ID ESTOQUE", "NOME PRODUTO", "CATEGORIA"]], on="ID ESTOQUE")
ranking_estoque_produtos = estoque_com_produtos.groupby(["NOME PRODUTO", "CATEGORIA"])["QTD ESTOQUE"].sum().reset_index()
ranking_estoque_produtos.rename(columns={"QTD ESTOQUE": "TOTAL_ESTOQUE"}, inplace=True)
ranking_estoque_produtos.sort_values(by="TOTAL_ESTOQUE", ascending=False, inplace=True)
top_estoque_produtos = ranking_estoque_produtos.head(20)
salvar_csv("questao_10_ranking_estoque_produtos", top_estoque_produtos)
plot_grafico(top_estoque_produtos,
             "Questão 10 - Top 20 Produtos por Quantidade de Estoque",
             "TOTAL_ESTOQUE", "NOME PRODUTO", "grafico_questao_10", orientacao='h')

print("\n✅ Análise concluída.")
print("✅ Tabelas salvas nos arquivos CSV.")
print("✅ Gráficos salvos como arquivos PNG.")


✅ Análise concluída.
✅ Tabelas salvas nos arquivos CSV.
✅ Gráficos salvos como arquivos PNG.


In [4]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os

# Função para carregar uma tabela do CSV
def carregar_tabela(nome_arquivo):
    df = pd.read_csv(nome_arquivo)
    return [list(df.columns)] + df.values.tolist()

# Função para formatar a tabela no PDF
def formata_tabela(tabela):
    styles = getSampleStyleSheet()
    return [[Paragraph(str(cell), styles["Normal"]) for cell in row] for row in tabela]

# Função para adicionar uma imagem ao relatório
def adicionar_imagem(nome_arquivo, width=400):
    if os.path.exists(nome_arquivo):
        return Image(nome_arquivo, width=width, height=250)
    else:
        return Paragraph(f"Imagem '{nome_arquivo}' não encontrada.", getSampleStyleSheet()["Normal"])

# Função para criar o relatório PDF
def criar_relatorio_pdf(nome_arquivo_saida):
    doc = SimpleDocTemplate(nome_arquivo_saida, pagesize=letter)
    elements = []

    # Estilos de texto
    styles = getSampleStyleSheet()
    title_style = styles["Title"]
    heading_style = styles["Heading2"]
    normal_style = styles["Normal"]

    # Título do relatório
    elements.append(Paragraph("Relatório Infomaz - Análise de Vendas", title_style))
    elements.append(Spacer(1, 24))

    # Adicionar conteúdo por seção
    elementos_secoes = [
        {
            "titulo": "Valor Total de Venda por Categoria",
            "tabela": "questao_1_valor_total_categoria.csv"
        },
        {
            "titulo": "Margem de Lucro por Produto",
            "tabela": "questao_2_margem_produtos.csv"
        },
        {
            "titulo": "Top 10 Clientes por Quantidade de Produtos Comprados por Mês",
            "tabela": "questao_3_ranking_clientes.csv",
            "imagem": "grafico_questao_3.png"
        },
        {
            "titulo": "Top 10 Fornecedores por Quantidade de Estoque Disponível por Mês",
            "tabela": "questao_4_ranking_fornecedores.csv"
        },
        {
            "titulo": "Top 10 Produtos por Quantidade de Venda por Mês",
            "tabela": "questao_5_ranking_produtos_quantidade.csv",
            "imagem": "grafico_questao_5.png"
        },
        {
            "titulo": "Top 10 Produtos por Valor de Venda por Mês",
            "tabela": "questao_6_ranking_produtos_valor.csv",
            "imagem": "grafico_questao_6.png"
        },
        {
            "titulo": "Média de Valor de Venda por Categoria por Mês",
            "tabela": "questao_7_media_valor_categoria.csv",
            "imagem": "grafico_questao_7.png"
        },
        {
            "titulo": "Média de Margem de Lucro por Categoria por Mês",
            "tabela": "questao_8_ranking_margem_categoria.csv",
            "imagem": "grafico_questao_8.png"
        },
        {
            "titulo": "Top 20 Produtos Mais Comprados por Clientes",
            "tabela": "questao_9_lista_produtos_clientes.csv"
        },
        {
            "titulo": "Top 20 Produtos por Quantidade de Estoque",
            "tabela": "questao_10_ranking_estoque_produtos.csv",
            "imagem": "grafico_questao_10.png"
        }
    ]

    for secao in elementos_secoes:
        # Adicionar título da seção
        elements.append(Paragraph(secao["titulo"], heading_style))
        elements.append(Spacer(1, 12))

        # Adicionar tabela
        if "tabela" in secao:
            tabela = carregar_tabela(secao["tabela"])
            tabela_formatada = formata_tabela(tabela)
            t = Table(tabela_formatada)
            t.setStyle(TableStyle([
                ('BACKGROUND', (0,0), (-1,0), colors.lightblue),
                ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
                ('ALIGN', (0,0), (-1,-1), 'CENTER'),
                ('FONTSIZE', (0,0), (-1,0), 10),
                ('BOTTOMPADDING', (0,0), (-1,0), 12),
                ('BACKGROUND', (0,1), (-1,-1), colors.beige),
                ('GRID', (0,0), (-1,-1), 1, colors.black)
            ]))
            elements.append(t)
        elements.append(Spacer(1, 12))

        # Adicionar imagem
        if "imagem" in secao:
            img = adicionar_imagem(secao["imagem"])
            elements.append(img)
        elements.append(Spacer(1, 24))

    # Build the PDF
    doc.build(elements)

    print(f"\n✅ Relatório PDF salvo como '{nome_arquivo_saida}'.")
    
criar_relatorio_pdf("relatorio_infomaz.pdf")


✅ Relatório PDF salvo como 'relatorio_infomaz.pdf'.
